# Relation Extraction
## Setup:
1. Download the **RelExtraction** Model and place it in the **Resources** Folder.
2. Create a Python virtual environment *(use: Pyenv, miniConda, etc..)*
3. Activate VirtualEnv.
4. Install needed python libraries *(python -m pip install -r requierments_rel.txt)*
5. Now you'll have an isolated sandbox to experiment around.

## Prediction
import relevant libraries

In [1]:
from Pipeline.pipeline_for_manual import MedicalIEPipeline
import re
from utils.document_helper import doc_to_brat
import os

Turn on the NER Server located under **NamedEntityRecognition/app.py** before starting. (Use the NER virtualEnv)

In [2]:
# Create MedicalPipeline to Apply NER, RelExtraction sequentially and use
# the results from NER model as input for the RelEx Model
med_pipeline = MedicalIEPipeline()

# Some Text as input
input = """Wegen Hypocalciaemie stationaer auf der 134 vom 22.7. bis 3.8.04 .
Gutes Befinden , keine Kraempfe , keine GI-Symptome , kein epigastrischen Schmerzen , Stuhlgang normal .
Letztes S-Calcium 1,94 unter orlaer Therapie mit Ca und Rocaltrol ."""

def normalize_text(text):
    return re.sub( '\\s+', ' ', text).strip()

# Normalize the input is not so important
text = normalize_text(input)

# Apply the Pipeline on the given text
doc = med_pipeline.get_annotated_document(text)

# Convert the generated documemt to Brat Format
brat = doc_to_brat(doc,
                       selected_ents=None,
                       selected_rels=None,
                       enable_negation=False,
                       enable_candidate_search=False,
                       enable_wsd=False)

/home/ammer/miniconda3/envs/RelEx/lib/python3.6/site-packages/flair/embeddings.py:217: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  word_embedding = torch.FloatTensor(word_embedding)


### Display Results
1. Text
2. Named-Entities
3. Relations among entities

In [3]:
# Text
print(brat['text'])
print('---------------------\n')

Wegen Hypocalciaemie stationaer auf der 134 vom 22.7. bis 3.8.04 . Gutes Befinden , keine Kraempfe , keine GI-Symptome , kein epigastrischen Schmerzen , Stuhlgang normal . Letztes S-Calcium 1,94 unter orlaer Therapie mit Ca und Rocaltrol .
---------------------



In [4]:
# Named-Entites
for ent in brat['entities']:
    print(ent[0]+'\t'+ent[1][:1]+ent[1].lower()[1:]+' '+str(ent[2][0][0])+' '+str(ent[2][0][1])+'\t'+ent[3])

T1	Medical_condition 6 20	Hypocalciaemie
T2	Treatment 21 31	stationaer
T3	Time_information 48 64	22.7. bis 3.8.04
T4	State_of_health 67 81	Gutes Befinden
T5	Medical_condition 90 98	Kraempfe
T6	Medical_condition 107 118	GI-Symptome
T7	Local_specification 126 140	epigastrischen
T8	Medical_condition 141 150	Schmerzen
T9	Process 153 162	Stuhlgang
T10	State_of_health 163 169	normal
T11	Biological_chemistry 180 189	S-Calcium
T12	Measurement 190 194	1,94
T13	Medication 201 207	orlaer
T14	Treatment 208 216	Therapie
T15	Medication 221 223	Ca
T16	Medication 228 237	Rocaltrol


In [5]:
# Relations:
for rel in brat['relations']:
    print(rel[0]+'\t'+rel[1].split('-')[1].lower()+' arg1:'+rel[2][0][1]+' arg2:'+rel[2][1][1])

R1	consequence arg1:T1 arg2:T2
R2	has_time_info arg1:T1 arg2:T3
R3	has_time_info arg1:T2 arg2:T3
R4	hasstate arg1:T5 arg2:T4
R5	hasstate arg1:T6 arg2:T4
R6	is_located arg1:T4 arg2:T7
R7	hasstate arg1:T8 arg2:T4
R8	hasstate arg1:T9 arg2:T4
R9	hasstate arg1:T10 arg2:T4
R10	hasstate arg1:T5 arg2:T6
R11	is_located arg1:T5 arg2:T7
R12	is_linked_to arg1:T5 arg2:T8
R13	hasstate arg1:T9 arg2:T5
R14	hasstate arg1:T5 arg2:T10
R15	is_located arg1:T6 arg2:T7
R16	is_linked_to arg1:T6 arg2:T8
R17	consequence arg1:T6 arg2:T9
R18	hasstate arg1:T6 arg2:T10
R19	is_located arg1:T8 arg2:T7
R20	is_located arg1:T9 arg2:T7
R21	is_located arg1:T10 arg2:T7
R22	hasstate arg1:T9 arg2:T8
R23	hasstate arg1:T8 arg2:T10
R24	hasstate arg1:T9 arg2:T10
R25	has_measure arg1:T11 arg2:T12
R26	treats arg1:T13 arg2:T11
R27	consequence arg1:T11 arg2:T14
R28	consequence arg1:T11 arg2:T15
R29	treats arg1:T16 arg2:T11
R30	has_measure arg1:T13 arg2:T12
R31	has_measure arg1:T14 arg2:T12
R32	consequence arg1:T12 arg2:T15
R33	has_m

## Background
### Pipeline Architecture
```python
class MedicalIEPipeline:
    _MEDICAL_IE_PIPELINE = None

    @staticmethod
    def get_pipeline():
        if MedicalIEPipeline._MEDICAL_IE_PIPELINE is None:
            # Create spacy pipeline
            pipeline = spacy.load('de_core_news_sm', disable=['tagger', 'ner', 'textcat'])

            # Create NER model
            named_entity_recognition = NER(pipeline)

            # Create REL model
            relation_extraction = RelationExtraction(pipeline, model_file='../Resources/relation_extraction_mex_model(default_word_relative_concept_embeddings).pt')

            # Add components to pipeline
            pipeline.add_pipe(named_entity_recognition)
            pipeline.add_pipe(relation_extraction)

            MedicalIEPipeline._MEDICAL_IE_PIPELINE = pipeline

        return MedicalIEPipeline._MEDICAL_IE_PIPELINE

```